# ASE & CP2K

Set some environment variables for the notebook. For doing this on a HPC resource, these would be set as export commands in the slurm script. Also, the ASE_CP2K_COMMAND would change to "srun cp2k_shell.psmp"

In [1]:
%set_env SHELL=/bin/bash
%set_env OMP_NUM_THREADS=8
%set_env ASE_CP2K_COMMAND=/home/woodrowwilson/Programs/cp2k/9.1/exe/PopOS-x86-64-gfortran/cp2k_shell.ssmp

env: SHELL=/bin/bash
env: OMP_NUM_THREADS=8
env: ASE_CP2K_COMMAND=/home/woodrowwilson/Programs/cp2k/9.1/exe/PopOS-x86-64-gfortran/cp2k_shell.ssmp


## Modules

In [2]:
# General
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# For building things
from ase import Atom, Atoms
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.build import molecule
from ase.visualize import view

# Unit Conversions and Fixing Atoms
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal
from ase.constraints import FixedPlane, FixedLine, FixAtoms

# ASE Calculators
from plumed import Plumed
from ase.calculators.cp2k import CP2K
from ase.calculators.lj import LennardJones
from ase.calculators.plumed import Plumed
from ase.calculators.idealgas import IdealGas

# Geometry Optimizations and Normal Mode Analysis
from ase.optimize import LBFGS, FIRE
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

# EOS fitting for Unit Cells
from ase.eos import EquationOfState

# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT

In [3]:
cwd = os.getcwd()
cwd

'/home/woodrowwilson/Documents/Projects/MSELProjects'

In [4]:
def myCP2KCalculator(tag, functional, ecut):
    """Creates a CP2K calculator object with Settings for Production Runs
    tag -> label for cp2k output
    functional -> Either PBE+D3, BEEFVDW, rVV10, or LDA+FermiDirac
    ecut -> PW Kinetic Energy Cutoff (Rydberg)
    """
    
    if functional == "PBE+D3":
        xc = "PBE"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &VDW_POTENTIAL
        POTENTIAL_TYPE PAIR_POTENTIAL
        &PAIR_POTENTIAL
          R_CUTOFF 15.0
          TYPE DFTD3
          CALCULATE_C9_TERM .FALSE.
          REFERENCE_FUNCTIONAL PBE
          PARAMETER_FILE_NAME dftd3.dat
        &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI ON
          OUTPUT_TEXT .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "BEEFVDW":
        xc = "GGA_XC_BEEFVDW"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
      &VDW_POTENTIAL
          &NON_LOCAL
              TYPE LMKLL
              KERNEL_FILE_NAME vdW_kernel_table.dat
          &END NON_LOCAL
      &END VDW_POTENTIAL
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI ON
          OUTPUT_TEXT .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "rVV10":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
       &XC_FUNCTIONAL
         &GGA_X_RPW86
         &END GGA_X_RPW86
         &GGA_C_PBE
         &END GGA_C_PBE
       &END XC_FUNCTIONAL
       &vdW_POTENTIAL
          DISPERSION_FUNCTIONAL NON_LOCAL
          &NON_LOCAL
            TYPE RVV10
            PARAMETERS 6.3 0.0093
            VERBOSE_OUTPUT
            KERNEL_FILE_NAME rVV10_kernel_table.dat
            CUTOFF 40
          &END NON_LOCAL
       &END vdW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI ON
          OUTPUT_TEXT .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "LDA+FermiDirac":
        xc = "LDA"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    &KPOINTS
        SCHEME MONKHORST-PACK 6 6 6
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI OFF
          OUTPUT_EMP .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "PBED3+FermiDirac":
        xc = "PBE"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    &KPOINTS
        SCHEME MONKHORST-PACK 6 6 6
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &VDW_POTENTIAL
        POTENTIAL_TYPE PAIR_POTENTIAL
        &PAIR_POTENTIAL
          R_CUTOFF 15.0
          TYPE DFTD3
          CALCULATE_C9_TERM .FALSE.
          REFERENCE_FUNCTIONAL PBE
          PARAMETER_FILE_NAME dftd3.dat
        &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI OFF
          OUTPUT_EMP .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    else:
        xc = "LDA"
        inp = ''''''
        
    calc = CP2K(
        auto_write=False,
        basis_set="DZVP-MOLOPT-SR-GTH",
        basis_set_file="BASIS_MOLOPT",
        charge=0,
        cutoff = ecut*Rydberg,
        debug = False,
        force_eval_method = "Quickstep",
        xc = xc,
        inp = inp,
        max_scf = 50,
        poisson_solver ="auto",
        potential_file = "POTENTIAL",
        pseudo_potential = "GTH-PBE",
        stress_tensor = True,
        print_level = "LOW",
        label = tag,
    )
    return calc

## Unit Cell Equation of State Fitting

For finding optimal lattice vectors, ASE can calculate the equation of state for a unit cell.

In [5]:
atoms = read("Resources/cif/tmc/Beta-Mo2C_mp-1221498_conventional_standard.cif")
view(atoms, viewer = "ngl")

In [6]:
dirname = "simulations_tmc_uc-eos_beta"
if not os.path.isdir(dirname):
    os.mkdir(dirname)
else:
    pass
os.chdir(dirname)

atoms.calc = myCP2KCalculator(dirname, "PBED3+FermiDirac", 400.0)

cell = atoms.get_cell()
traj = Trajectory('eos.traj', 'w')
for x in np.linspace(0.95, 1.05, 5):
    print(cell * x)
    atoms.set_cell(cell * x, scale_atoms=True)
    atoms.get_potential_energy()
    traj.write(atoms)

os.chdir(cwd)

configs = read('simulations_tmc_uc-eos_beta/eos.traj@0:5')  # read 5 configurations
# Extract volumes and energies:
volumes = [atoms.get_volume() for atoms in configs]
energies = [atoms.get_potential_energy() for atoms in configs]
eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()

s = (v0/cell.volume)**(1./3.)
atoms.set_cell(cell*s, scale_atoms=True)
print("Scalar: ", s)
print("Evaluating at Minimum:", atoms.get_potential_energy())
print("Bulk Modulus [GPa]:", B / kJ * 1.0e24)

eos.plot()

[[ 2.91944646  0.          0.        ]
 [-1.45972323  2.5283148   0.        ]
 [ 0.          0.          4.41991057]]
[[ 2.996274   0.         0.       ]
 [-1.498137   2.5948494  0.       ]
 [ 0.         0.         4.536224 ]]
[[ 3.07310154  0.          0.        ]
 [-1.53655077  2.661384    0.        ]
 [ 0.          0.          4.65253744]]
[[ 3.14992908  0.          0.        ]
 [-1.57496454  2.7279186   0.        ]
 [ 0.          0.          4.76885088]]
[[ 3.22675662  0.          0.        ]
 [-1.61337831  2.7944532   0.        ]
 [ 0.          0.          4.88516431]]


In [25]:
atoms = read("Resources/cif/tmc/Alpha-Mo2C_mp-1552_conventional_standard.cif")
view(atoms, viewer = "ngl")

In [ ]:
dirname = "simulations_tmc_uc-eos_alpha"
if not os.path.isdir(dirname):
    os.mkdir(dirname)
else:
    pass
os.chdir(dirname)

atoms.calc = myCP2KCalculator(dirname, "PBED3+FermiDirac", 400.0)

cell = atoms.get_cell()
traj = Trajectory('eos.traj', 'w')
for x in np.linspace(0.95, 1.05, 5):
    print(cell * x)
    atoms.set_cell(cell * x, scale_atoms=True)
    atoms.get_potential_energy()
    traj.write(atoms)

os.chdir(cwd)

configs = read('simulations_tmc_uc-eos_alpha/eos.traj@0:5')  # read 5 configurations
# Extract volumes and energies:
volumes = [atoms.get_volume() for atoms in configs]
energies = [atoms.get_potential_energy() for atoms in configs]
eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()

s = (v0/cell.volume)**(1./3.)
atoms.set_cell(cell*s, scale_atoms=True)
print("Scalar: ", s)
print("Evaluating at Minimum:", atoms.get_potential_energy())
print("Bulk Modulus [GPa]:", B / kJ * 1.0e24)

eos.plot()